<a href="https://colab.research.google.com/github/jovishacurlie22/SCLPackage/blob/main/SCLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#A menudriven program to showcase cryptographic techniques
!pip install pycryptodome
import random
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64
from sympy import isprime, mod_inverse

# Function for Caesar Cipher encryption
def caesar_cipher(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            shift_amount = shift % 26
            new_char = chr((ord(char) - ord('A') + shift_amount) % 26 + ord('A')) if char.isupper() else chr((ord(char) - ord('a') + shift_amount) % 26 + ord('a'))
            result += new_char
        else:
            result += char
    return result

# Function for Caesar Cipher decryption
def caesar_decrypt(text, shift):
    return caesar_cipher(text, -shift)  # Decrypting is just reversing the shift

# Function for Vigenère Cipher encryption
def vigenere_cipher(text, key):
    key = key.lower()
    key_length = len(key)
    result = ""
    for i, char in enumerate(text):
        if char.isalpha():
            shift = ord(key[i % key_length]) - ord('a')
            new_char = chr((ord(char) - ord('A') + shift) % 26 + ord('A')) if char.isupper() else chr((ord(char) - ord('a') + shift) % 26 + ord('a'))
            result += new_char
        else:
            result += char
    return result

# Function for Vigenère Cipher decryption
def vigenere_decrypt(text, key):
    key = key.lower()
    key_length = len(key)
    result = ""
    for i, char in enumerate(text):
        if char.isalpha():
            shift = ord(key[i % key_length]) - ord('a')
            new_char = chr((ord(char) - ord('A') - shift) % 26 + ord('A')) if char.isupper() else chr((ord(char) - ord('a') - shift) % 26 + ord('a'))
            result += new_char
        else:
            result += char
    return result

# Function to generate a random prime number
def generate_prime(bits):
    while True:
        prime_candidate = random.getrandbits(bits)
        if isprime(prime_candidate):
            return prime_candidate

# Function to generate RSA keys
def generate_rsa_keys(bits=8):
    p = generate_prime(bits)
    q = generate_prime(bits)

    n = p * q
    phi_n = (p - 1) * (q - 1)

    # Choose e such that 1 < e < phi_n and e is coprime to phi_n
    e = 3
    while e < phi_n:
        if gcd(e, phi_n) == 1:
            break
        e += 2  # Increment by 2 to ensure e is odd

    d = mod_inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Function to calculate gcd
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# Function to encrypt a message using AES
def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_CBC)
    iv = cipher.iv
    padded_plaintext = pad(plaintext).encode('utf-8')
    ciphertext = cipher.encrypt(padded_plaintext)
    encrypted_message = base64.b64encode(iv + ciphertext).decode('utf-8')
    return encrypted_message

# Function to decrypt a message using AES
def aes_decrypt(encrypted_message, key):
    encrypted_message = base64.b64decode(encrypted_message)
    iv = encrypted_message[:16]
    ciphertext = encrypted_message[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_padded = cipher.decrypt(ciphertext)
    return unpad(decrypted_padded.decode('utf-8'))

# Function to pad the plaintext for AES
def pad(text):
    padding_length = 16 - (len(text) % 16)
    return text + (chr(padding_length) * padding_length)

# Function to unpad the plaintext for AES
def unpad(padded_text):
    padding_length = ord(padded_text[-1])
    return padded_text[:-padding_length]

# Function to decrypt a message using RSA
def rsa_decrypt(ciphertext, private_key):
    n, d = private_key
    message_int = pow(ciphertext, d, n)
    message_length = (message_int.bit_length() + 7) // 8
    message = message_int.to_bytes(message_length, 'big').decode('utf-8', 'ignore')
    return message

# Function to encrypt a message using RSA
def rsa_encrypt(message, public_key):
    n, e = public_key
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    ciphertext = pow(message_int, e, n)
    return ciphertext

# Main program
def main():
    while True:
        print("\nMenu:")
        print("1. Caesar Cipher")
        print("2. Vigenère Cipher")
        print("3. AES Encryption")
        print("4. RSA Encryption")
        print("5. Exit")

        choice = input("Choose an option (1-5): ")

        if choice == '1':
            message = input("Enter the message: ")
            shift = int(input("Enter the shift value (1-25): "))
            encrypted_message = caesar_cipher(message, shift)
            decrypted_message = caesar_decrypt(encrypted_message, shift)
            print(f"Encrypted Message (Caesar Cipher): {encrypted_message}")
            print(f"Decrypted Message (Caesar Cipher): {decrypted_message}")

        elif choice == '2':
            message = input("Enter the message: ")
            key = input("Enter the key: ")
            encrypted_message = vigenere_cipher(message, key)
            decrypted_message = vigenere_decrypt(encrypted_message, key)
            print(f"Encrypted Message (Vigenère Cipher): {encrypted_message}")
            print(f"Decrypted Message (Vigenère Cipher): {decrypted_message}")

        elif choice == '3':
            key = get_random_bytes(16)  # AES-128
            message = input("Enter the message: ")
            encrypted_message = aes_encrypt(message, key)
            decrypted_message = aes_decrypt(encrypted_message, key)
            print(f"Encrypted Message (AES): {encrypted_message}")
            print(f"Key (AES): {base64.b64encode(key).decode('utf-8')}")
            print(f"Decrypted Message (AES): {decrypted_message}")

        elif choice == '4':
            bits = 32
            public_key, private_key = generate_rsa_keys(bits)
            message = input("Enter the message: ")
            ciphertext = rsa_encrypt(message, public_key)
            decrypted_message = rsa_decrypt(ciphertext, private_key)
            print(f"Ciphertext (RSA): {ciphertext}")
            print(f"Decrypted Message (RSA): {decrypted_message}")

        elif choice == '5':
            print("Exiting...")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
